In [1]:
import numpy as np
import pyramid
import pandas as pd
import math
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
import requests
from io import BytesIO
from sklearn import metrics
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Legend
import pandas as pd
from bokeh.models import Span
from sklearn.metrics import mean_absolute_error

### Data

In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
df = pd.read_csv("AirPassengers.csv")

data_train = np.array(df.iloc[:100]['#Passengers'])
data_test = np.array(df.iloc[100:]['#Passengers'])

#### Plotting

In [4]:
output_notebook()

def plot_arima(truth, forecasts, title="ARIMA", xaxis_label='Time',
               yaxis_label='Value', c1='#A6CEE3', c2='#B2DF8A', 
               forecast_start=None, **kwargs):
    
    # make truth and forecasts into pandas series
    n_truth = truth.shape[0]
    n_forecasts = forecasts.shape[0]
    
    # always plot truth the same
    truth = pd.Series(truth, index=np.arange(truth.shape[0]))
    
    # if no defined forecast start, start at the end
    if forecast_start is None:
        idx = np.arange(n_truth, n_truth + n_forecasts)
    else:
        idx = np.arange(forecast_start, n_forecasts)
    forecasts = pd.Series(forecasts, index=idx)
    
    # set up the plot
    p = figure(title=title, plot_height=400, **kwargs)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    
    # add the lines
    p.line(forecasts.index, forecasts.values, color=c2, legend='Forecasted')
    p.line(truth.index, truth.values, color=c1, legend='Observed')
    #vline = Span(location=49,dimension='height', line_color='red',line_width=1)
    #p.renderers.extend([vline])
    
    return p

Loading BokehJS ...

### Pyramid auto arima

In [5]:
from pyramid.arima import auto_arima

rs_fit = auto_arima(data_train, start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                    start_P=0, seasonal=True, n_jobs=-1, d=1, D=1, trace=True,
                    error_action='ignore',  # don't want to know if an order does not work
                    suppress_warnings=True,  # don't want convergence warnings
                    stepwise=False, random=True, random_state=42,  # we can fit a random search (not exhaustive)
                    n_fits=100)

rs_fit.summary()

Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(3, 1, 3) seasonal_order=(0, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(2, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 3) seasonal_order=(2, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(3, 1, 2) seasonal_order=(1, 1, 1, 12); AIC=648.172, BIC=670.366, Fit time=2.235 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(3, 1, 3) seasonal_order=(0, 1, 1, 12); AIC=646.242, BIC=668.435, Fit time=2.690 seconds
Fit ARIMA: order=(1, 1, 3) seasonal_order=(0, 1, 1, 12); AIC=644.247, BIC=661.508, Fit time=2.806 seconds
Fit ARIMA: order=(2, 1, 3) seasonal_order=(1, 1, 1, 12); AIC=649.494, BIC=671.688, Fit time=2.130 seconds
Fit ARIMA: order=(1, 1, 3) seasonal_order=(0, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  100
Model:             SARIMAX(1, 1, 3)x(0, 1, 1, 12)   Log Likelihood                -315.123
Date:                            Mon, 09 Jul 2018   AIC                            644.247
Time:                                    19:12:07   BIC                            661.508
Sample:                                         0   HQIC                           651.197
                                            - 100                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.1223      0.075      1.641      0.101      -0.024       0.268
ar.L1          0.5885      0.206      2.857      0.004       0.185       0.992
ma.L1         -0.8936      7.531     -0.119      0.906     -15.654      13.867
ma.L2          0.1739      0.768      0.226      0.821      -1.331       1.679
ma.L3         -0.2799      2.025     -0.138      0.890      -4.249       3.689
ma.S.L12      -0.2355      0.132     -1.784      0.074      -0.494       0.023
sigma2        78.1878    577.195      0.135      0.892   -1053.093    1209.469
===================================================================================
Ljung-Box (Q):                       30.50   Jarque-Bera (JB):                 1.20
Prob(Q):                              0.86   Prob(JB):                         0.55
Heteroskedasticity (H):               1.21   Skew:                             0.25
Prob(H) (two-sided):                  0.61   Kurtosis:                         3.27
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [6]:
in_sample_preds = rs_fit.predict_in_sample()
#predicting on test(validate here)
next_validate = rs_fit.predict(n_periods=44)
print ("MAPE on train: ",mean_absolute_percentage_error(data_train, in_sample_preds))

MAPE on train:  5.12192938472019


In [7]:
show(plot_arima(data_train, in_sample_preds, 
                title="Original Series & In-sample Predictions", 
                c2='#FF0000', forecast_start=0))

In [8]:
show(plot_arima(data_train, next_validate, 
                title="Original Series & In-sample Predictions", 
                c2='#FF0000'))

In [9]:
full_data = np.concatenate([data_train,data_test])
full_pred = np.concatenate([in_sample_preds,next_validate])
show(plot_arima(full_data, full_pred, 
                title="Original Series & In-sample Predictions", 
                c2='#FF0000', forecast_start = 0))

In [10]:
mean_absolute_percentage_error(data_test,next_validate)

8.58923500338057

In [11]:
mean_absolute_error(data_test, next_validate)

34.2154559324368

### PyAF

In [12]:
import pyaf.ForecastEngine as autof

In [13]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [14]:
df['Month'] = df['Month'].apply(lambda x : datetime.strptime(x, "%Y-%m"))
data_train_for_pyfa = df.iloc[:100]

In [15]:
lEngine = autof.cForecastEngine()
lEngine.train(data_train_for_pyfa, 'Month' , '#Passengers',44);
lEngine.getModelInfo()

INFO:pyaf.std:START_TRAINING '#Passengers'
INFO:pyaf.std:END_TRAINING_TIME_IN_SECONDS '#Passengers' 6.137075901031494
INFO:pyaf.std:TIME_DETAIL TimeVariable='Month' TimeMin=1949-01-01T00:00:00.000000 TimeMax=1957-04-01T00:00:00.000000 TimeDelta=<DateOffset: months=1> Estimation = (0 , 100) Validation = (0 , 100) Test = (0 , 100) Horizon=44
INFO:pyaf.std:SIGNAL_DETAIL_ORIG SignalVariable='#Passengers' Min=104 Max=413  Mean=218.36 StdDev=73.84842855470926
INFO:pyaf.std:SIGNAL_DETAIL_TRANSFORMED TransformedSignalVariable='_#Passengers' Min=104 Max=413  Mean=218.36 StdDev=73.84842855470926
INFO:pyaf.std:BEST_TRANSOFORMATION_TYPE '_'
INFO:pyaf.std:BEST_DECOMPOSITION  '_#Passengers_ConstantTrend_residue_zeroCycle_residue_AR(25)' [ConstantTrend + NoCycle + AR(25)]
INFO:pyaf.std:TREND_DETAIL '_#Passengers_ConstantTrend' [ConstantTrend]
INFO:pyaf.std:CYCLE_DETAIL '_#Passengers_ConstantTrend_residue_zeroCycle' [NoCycle]
INFO:pyaf.std:AUTOREG_DETAIL '_#Passengers_ConstantTrend_residue_zeroCycle_r

In [16]:
forecast_df = lEngine.forecast(data_train_for_pyfa, 44);
forecast_df[['Month' , '#Passengers' , '#Passengers_Forecast']].tail(50)

INFO:pyaf.std:START_FORECASTING
INFO:pyaf.std:END_FORECAST_TIME_IN_SECONDS 4.046411752700806


,Month,#Passengers,#Passengers_Forecast
94,1956-11-01,271.0,268.288533
95,1956-12-01,306.0,309.786446
96,1957-01-01,315.0,315.519205
97,1957-02-01,301.0,304.622571
98,1957-03-01,356.0,346.405896
99,1957-04-01,348.0,352.654469
100,1957-05-01,NaN,361.630634
101,1957-06-01,NaN,428.146049
102,1957-07-01,NaN,473.171720
103,1957-08-01,NaN,461.127390


In [17]:
predictions_pyfa = np.array(forecast_df['#Passengers_Forecast'])
print (predictions_pyfa)

[117.45436747 117.45436747 121.68573519 131.7483902  129.84614316
 123.24823143 134.03549895 147.69935802 145.26275307 132.27456497
 124.65285161 118.11956552 123.45062215 123.84425236 140.24072845
 136.27703211 130.31139453 139.10642682 161.99134412 167.53618086
 161.60870473 136.19097323 123.078418   128.32549613 140.80268564
 150.19011751 170.03949932 161.59500758 157.69616121 190.94701611
 202.24338035 195.63076714 187.01157159 156.8688426  137.38244657
 164.99518461 168.1396762  175.33845847 201.82018473 184.15515485
 186.72296203 203.86638761 233.90750472 230.8924665  211.4196715
 181.03920507 173.11044825 193.24850909 199.98798945 203.81145985
 218.36523526 215.21521525 236.93680318 254.84987104 262.76371687
 269.81304549 252.47127074 206.09920287 194.35342059 202.70718485
 216.06765989 200.9417938  237.60804826 236.7154882  238.08951506
 260.65788643 290.05209878 304.40972319 258.73175558 228.47935716
 203.6790948  232.22659683 232.29870335 228.518754   279.86277043
 268.893935

In [18]:
forecast_df

,Month,#Passengers,_#Passengers,row_number,Month_Normalized,_#Passengers_ConstantTrend,_#Passengers_ConstantTrend_residue,_#Passengers_ConstantTrend_residue_zeroCycle,_#Passengers_ConstantTrend_residue_zeroCycle_residue,_#Passengers_ConstantTrend_residue_zeroCycle_residue_AR(25),...,_#Passengers_Cycle,_#Passengers_Cycle_residue,_#Passengers_AR,_#Passengers_AR_residue,_#Passengers_TransformedForecast,#Passengers_Forecast,_#Passengers_TransformedResidue,#Passengers_Residue,#Passengers_Forecast_Lower_Bound,#Passengers_Forecast_Upper_Bound
0,1949-01-01,112.0,112.0,0,0.000000,218.36,-106.36,0.0,-106.36,-100.905633,...,0.0,-106.36,-100.905633,-5.454367,117.454367,117.454367,-5.454367,-5.454367,NaN,NaN
1,1949-02-01,118.0,118.0,1,0.010292,218.36,-100.36,0.0,-100.36,-100.905633,...,0.0,-100.36,-100.905633,0.545633,117.454367,117.454367,0.545633,0.545633,NaN,NaN
2,1949-03-01,132.0,132.0,2,0.019588,218.36,-86.36,0.0,-86.36,-96.674265,...,0.0,-86.36,-96.674265,10.314265,121.685735,121.685735,10.314265,10.314265,NaN,NaN
3,1949-04-01,129.0,129.0,3,0.029880,218.36,-89.36,0.0,-89.36,-86.611610,...,0.0,-89.36,-86.611610,-2.748390,131.748390,131.748390,-2.748390,-2.748390,NaN,NaN
4,1949-05-01,121.0,121.0,4,0.039841,218.36,-97.36,0.0,-97.36,-88.513857,...,0.0,-97.36,-88.513857,-8.846143,129.846143,129.846143,-8.846143,-8.846143,NaN,NaN
5,1949-06-01,135.0,135.0,5,0.050133,218.36,-83.36,0.0,-83.36,-95.111769,...,0.0,-83.36,-95.111769,11.751769,123.248231,123.248231,11.751769,11.751769,NaN,NaN
6,1949-07-01,148.0,148.0,6,0.060093,218.36,-70.36,0.0,-70.36,-84.324501,...,0.0,-70.36,-84.324501,13.964501,134.035499,134.035499,13.964501,13.964501,NaN,NaN
7,1949-08-01,148.0,148.0,7,0.070385,218.36,-70.36,0.0,-70.36,-70.660642,...,0.0,-70.36,-70.660642,0.300642,147.699358,147.699358,0.300642,0.300642,NaN,NaN
8,1949-09-01,136.0,136.0,8,0.080677,218.36,-82.36,0.0,-82.36,-73.097247,...,0.0,-82.36,-73.097247,-9.262753,145.262753,145.262753,-9.262753,-9.262753,NaN,NaN
9,1949-10-01,119.0,119.0,9,0.090637,218.36,-99.36,0.0,-99.36,-86.085435,...,0.0,-99.36,-86.085435,-13.274565,132.274565,132.274565,-13.274565,-13.274565,NaN,NaN


In [21]:
show(plot_arima(full_data, predictions_pyfa,
            title="Original Series & In-sample Predictions",
            c2='#FF0000', forecast_start=0))

In [22]:
print ("MAPE PyFA:",mean_absolute_percentage_error(data_test,predictions_pyfa[100:]))
print ("MAE  PyFA:",mean_absolute_error(data_test,predictions_pyfa[100:]))

MAPE PyFA: 12.006545969674141
MAE  PyFA: 51.966111804084555
